In [1]:
from collections import Counter
from flask import Flask, request, render_template, jsonify
import pickle
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
import argparse
import time
from splinter import Browser
import time
from collections import defaultdict
import json
import re
import requests
import boto3
from boto3.dynamodb.conditions import Key, Attr
import json
from decimal import Decimal

In [2]:
%load_ext autoreload
%autoreload 2

In [121]:
from src.cleaner import Cleaner 
import src.fantasy_scoring as fs
from src.db_helper import DB_helper

In [101]:
f = open('.env','r')
env = {}
print('reading env variables')
for line in f.readlines():
    env[line.strip().split("=")[0]]=line.strip().split("=")[1]
dynamodb = boto3.resource('dynamodb', region_name='us-east-1',aws_access_key_id = env['aws_access_key_id'],
    aws_secret_access_key = env['aws_secret_access_key'])
table = dynamodb.Table('nba_bbref')

reading env variables


In [122]:
clnr = Cleaner()
db = DB_helper()

reading env variables


In [13]:
lag = "_06"
lag_cols = [col+lag for col in clnr.lag_cols]

In [14]:
lag_cols

['minutes_06',
 'dk_score_06',
 'dk_per_min_06',
 'dk_no_fg_06',
 'dk_no_fg_per_min_06',
 'assists_06',
 'assists_per_min_06',
 'blocks_06',
 'blocks_per_min_06',
 'or_per_min_06',
 'dr_per_min_06',
 'steals_per_min_06',
 'turnovers_per_min_06',
 'doubles_06',
 'fg_pct_06',
 'ft_pct_06',
 'two_pct_06',
 'three_pct_06',
 'game_score_06',
 'attempted_field_goals_06',
 'attempted_three_point_field_goals_06',
 'w_06']

In [72]:
slug = 'curryst01'
resp = db.get_player(slug,lag_cols)
print(len(resp))


595


In [75]:
resp.head()

,assists_06,assists_per_min_06,attempted_field_goals_06,attempted_three_point_field_goals_06,blocks_06,blocks_per_min_06,date,dk_no_fg_06,dk_no_fg_per_min_06,dk_per_min_06,dk_score_06,doubles_06,dr_per_min_06,fg_pct_06,ft_pct_06,game_score_06,minutes_06,or_per_min_06,slug,steals_per_min_06,three_pct_06,turnovers_per_min_06,two_pct_06,w_06
0,8,0.185,16,8,0,0,2013-01-02,21.5,0.498,1.286,55.5,1,0.116,0.688,0.75,28.4,43.15,0.023,curryst01,0.046,0.75,0.093,0.625,1
1,6.5,0.179,13.5,6.5,0,0,2013-01-05,14,0.361,0.936,36.25,1,0.058,0.526,0.875,16.9,36.075,0.012,curryst01,0.023,0.475,0.081,0.562,0.5
2,5.667,0.152,16.333,7,0,0,2013-01-09,13.667,0.348,0.945,37.167,1,0.063,0.502,0.583,15.867,37.556,0.016,curryst01,0.032,0.483,0.087,0.518,0.333
3,7.25,0.183,17.75,7.25,0.25,0.006,2013-01-11,17.938,0.437,1.02,41.438,1.25,0.065,0.456,0.688,17.15,39.067,0.012,curryst01,0.041,0.456,0.071,0.46,0.5
4,7.2,0.18,17.8,7.6,0.2,0.005,2013-01-13,16.75,0.408,1.033,42.05,1.2,0.052,0.487,0.55,18.54,39.457,0.01,curryst01,0.038,0.521,0.062,0.457,0.4


In [22]:
data1 = pd.read_csv("data/bb_reference/2016to2019Data_merged.csv", encoding = 'latin-1')
data2 = pd.read_csv("data/bb_reference/2013to2015Data_merged.csv", encoding = 'latin-1')
data = data1.append(data2)
data = data.drop(['Unnamed: 0_y','Unnamed: 0'], axis = 1)
data = data.rename(columns={'Unnamed: 0_x':'gamescore_rank'})
data = clnr.transform(data)
data['date']=data['dt'].apply(lambda x: x.strftime("%Y-%m-%d"))
players = data['slug'].unique()
names = data['player'].unique()
slug_names = {}
name_slugs = {}
for row in data[['slug','player']].groupby('slug').agg('first').iterrows():
    slug_names[row[0]]=row[1]['player']
    name_slugs[row[1]['player']]=row[0]

In [80]:
test_ = data.iloc[0:100].copy(deep = True)
test_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 51
Data columns (total 54 columns):
gamescore_rank                       100 non-null int64
assists                              100 non-null int64
attempted_field_goals                100 non-null int64
attempted_free_throws                100 non-null int64
attempted_three_point_field_goals    100 non-null int64
blocks                               100 non-null int64
date                                 100 non-null object
defensive_rebounds                   100 non-null int64
game_score                           100 non-null float64
location                             100 non-null object
made_field_goals                     100 non-null int64
made_free_throws                     100 non-null int64
made_three_point_field_goals         100 non-null int64
name                                 100 non-null object
offensive_rebounds                   100 non-null int64
opponent                             100 non-null o

In [135]:
data = data.sort_values(by='date',axis = 0)
data.head()

,gamescore_rank,assists,attempted_field_goals,attempted_free_throws,attempted_three_point_field_goals,blocks,date,defensive_rebounds,game_score,location,made_field_goals,made_free_throws,made_three_point_field_goals,name,offensive_rebounds,opponent,outcome,personal_fouls,seconds_played,slug,steals,team,turnovers,player,birthday,year_min,year_max,pos,height,weight,points,rebounds,doubles,dd,td,dk_score,dk_per_min,dk_no_fg,dk_no_fg_per_min,assists_per_min,blocks_per_min,or_per_min,dr_per_min,steals_per_min,turnovers_per_min,points_per_min,double_per_min,minutes,fg_pct,w,three_pct,two_pct,ft_pct,dt
0,22,3,17,0,0,0,2013-01-01,3,13.7,Location.AWAY,10,0,0,Al Horford,0,Team.NEW_ORLEANS_HORNETS,Outcome.WIN,1,2467,horfoal01,0,Team.ATLANTA_HAWKS,1,Al Horford,3-Jun-86,2008,2019,C-F,10-Jun,245.0,20,3,1,0,0,27.75,0.674909,7.75,0.188488,0.072963,0.000000,0.00000,0.072963,0.000000,0.024321,0.486421,0.000000,41.116667,0.588235,1,0.000000,0.588235,0.000000,2013-01-01
60,35,1,11,0,0,3,2013-01-01,7,10.0,Location.AWAY,6,0,0,Chris Kaman,0,Team.WASHINGTON_WIZARDS,Outcome.WIN,4,1563,kamanch01,1,Team.DALLAS_MAVERICKS,1,Chris Kaman,28-Apr-82,2004,2016,C,Jul-00,265.0,12,7,1,0,0,29.75,1.142035,17.75,0.681382,0.038388,0.115163,0.00000,0.268714,0.038388,0.038388,0.460653,0.038388,26.050000,0.545455,1,0.000000,0.545455,0.000000,2013-01-01
87,104,1,2,0,0,0,2013-01-01,2,-0.1,Location.HOME,0,0,0,Al-Farouq Aminu,0,Team.ATLANTA_HAWKS,Outcome.LOSS,0,326,aminual01,0,Team.NEW_ORLEANS_HORNETS,0,Al-Farouq Aminu,21-Sep-90,2011,2019,F,9-Jun,220.0,0,2,0,0,0,4.00,0.736196,4.00,0.736196,0.184049,0.000000,0.00000,0.368098,0.000000,0.000000,0.000000,0.000000,5.433333,0.000000,0,0.000000,0.000000,0.000000,2013-01-01
86,8,2,21,2,0,0,2013-01-01,6,17.7,Location.AWAY,10,1,0,DeMarcus Cousins,8,Team.DETROIT_PISTONS,Outcome.LOSS,5,2142,couside01,3,Team.SACRAMENTO_KINGS,2,DeMarcus Cousins,13-Aug-90,2011,2019,C-F,11-Jun,270.0,21,14,2,1,0,48.00,1.344538,27.00,0.756303,0.056022,0.000000,0.22409,0.168067,0.084034,0.056022,0.588235,0.084034,35.700000,0.476190,0,0.000000,0.476190,0.500000,2013-01-01
85,4,5,14,7,3,1,2013-01-01,12,23.3,Location.AWAY,8,4,2,Evan Turner,1,Team.LOS_ANGELES_LAKERS,Outcome.WIN,1,2455,turneev01,2,Team.PHILADELPHIA_76ERS,1,Evan Turner,27-Oct-88,2011,2019,G,7-Jun,220.0,22,13,2,1,0,53.75,1.313646,30.75,0.751527,0.122200,0.024440,0.02444,0.293279,0.048880,0.024440,0.537678,0.048880,40.916667,0.571429,1,0.666667,0.545455,0.571429,2013-01-01


In [149]:
out = clnr.time_stats(test)
#out['date']=test['dt'].apply(lambda x: x.strftime("%Y-%m-%d"))
out['date']=test['date'].values
out =out.drop(labels = ['dt'],axis = 1)
#out['date']=1
out.head()
#out[0].merge(out[1],on='dt',suffixes = ('_sum','_avg'))

,attempted_field_goals_07_sum,dk_score_07_sum,dk_no_fg_07_sum,minutes_07_sum,rebounds_07_sum,fg_pct_07_avg,three_pct_07_avg,ft_pct_07_avg,minutes_07_avg,rebounds_07_avg,w_07_avg,date
0,0.0,0.00,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,2013-10-30
1,1.0,7.25,5.25,18.450000,3.0,1.000000,0.0,0.000000,18.450000,3.0,1.000000,2013-11-01
2,6.0,28.75,18.75,41.333333,11.0,0.700000,0.0,0.333333,20.666667,5.5,0.500000,2013-11-03
3,9.0,37.25,25.25,50.116667,15.0,0.577778,0.0,0.222222,16.705556,5.0,0.666667,2013-11-06
4,11.0,53.25,37.25,51.833333,21.0,0.466667,0.0,0.555556,17.277778,7.0,0.666667,2013-11-08


In [95]:
test = data [data['slug']=='adamsst01']
test.head()

,gamescore_rank,assists,attempted_field_goals,attempted_free_throws,attempted_three_point_field_goals,blocks,date,defensive_rebounds,game_score,location,made_field_goals,made_free_throws,made_three_point_field_goals,name,offensive_rebounds,opponent,outcome,personal_fouls,seconds_played,slug,steals,team,turnovers,player,birthday,year_min,year_max,pos,height,weight,points,rebounds,doubles,dd,td,dk_score,dk_per_min,dk_no_fg,dk_no_fg_per_min,assists_per_min,blocks_per_min,or_per_min,dr_per_min,steals_per_min,turnovers_per_min,points_per_min,double_per_min,minutes,fg_pct,w,three_pct,two_pct,ft_pct,dt
18249,200,1,1,1,0,0,2013-10-30,1,2.5,Location.AWAY,1,0,0,Steven Adams,2,Team.UTAH_JAZZ,Outcome.WIN,3,1107,adamsst01,0,Team.OKLAHOMA_CITY_THUNDER,0,Steven Adams,20-Jul-93,2014,2019,C,Jul-00,265.0,2,3,0,0,0,7.25,0.392954,5.25,0.284553,0.054201,0.000000,0.108401,0.054201,0.000000,0.000000,0.108401,0.000000,18.450000,1.000000,1,0.0,1.000000,0.000000,2013-10-30
18571,100,1,5,6,0,1,2013-11-01,3,8.3,Location.AWAY,2,4,0,Steven Adams,5,Team.MINNESOTA_TIMBERWOLVES,Outcome.LOSS,5,1373,adamsst01,0,Team.OKLAHOMA_CITY_THUNDER,0,Steven Adams,20-Jul-93,2014,2019,C,Jul-00,265.0,8,8,0,0,0,21.50,0.939548,13.50,0.589949,0.043700,0.043700,0.218500,0.131100,0.000000,0.000000,0.349599,0.000000,22.883333,0.400000,0,0.0,0.400000,0.666667,2013-11-01
18885,95,0,3,0,0,1,2013-11-03,0,2.0,Location.HOME,1,0,0,Steven Adams,4,Team.PHOENIX_SUNS,Outcome.WIN,2,527,adamsst01,0,Team.OKLAHOMA_CITY_THUNDER,1,Steven Adams,20-Jul-93,2014,2019,C,Jul-00,265.0,2,4,0,0,0,8.50,0.967742,6.50,0.740038,0.000000,0.113852,0.455408,0.000000,0.000000,0.113852,0.227704,0.000000,8.783333,0.333333,1,0.0,0.333333,0.000000,2013-11-03
19365,57,0,3,2,0,1,2013-11-06,7,10.5,Location.HOME,2,2,0,Steven Adams,2,Team.DALLAS_MAVERICKS,Outcome.WIN,1,1210,adamsst01,2,Team.OKLAHOMA_CITY_THUNDER,0,Steven Adams,20-Jul-93,2014,2019,C,Jul-00,265.0,6,9,0,0,0,23.25,1.152893,17.25,0.855372,0.000000,0.049587,0.099174,0.347107,0.099174,0.000000,0.297521,0.099174,20.166667,0.666667,1,0.0,0.666667,1.000000,2013-11-06
19679,12,3,10,4,0,3,2013-11-08,5,20.6,Location.AWAY,7,3,0,Steven Adams,5,Team.DETROIT_PISTONS,Outcome.WIN,5,1831,adamsst01,1,Team.OKLAHOMA_CITY_THUNDER,0,Steven Adams,20-Jul-93,2014,2019,C,Jul-00,265.0,17,10,2,1,0,43.50,1.425451,26.50,0.868378,0.098307,0.098307,0.163845,0.163845,0.032769,0.000000,0.557073,0.032769,30.516667,0.700000,1,0.0,0.700000,0.750000,2013-11-08


In [99]:
offset = 7
time_lag_sums=['dt','made_field_goals','dk_score','minutes','rebounds']
test[time_lag_sums].rolling('7d', on = 'dt',closed = 'left').sum().fillna(0)

,dt,made_field_goals,dk_score,minutes,rebounds
18249,2013-10-30,0.0,0.00,0.000000,0.0
18571,2013-11-01,1.0,7.25,18.450000,3.0
18885,2013-11-03,3.0,28.75,41.333333,11.0
19365,2013-11-06,4.0,37.25,50.116667,15.0
19679,2013-11-08,5.0,53.25,51.833333,21.0
19970,2013-11-10,10.0,75.25,59.466667,23.0
20492,2013-11-13,9.0,73.75,70.250000,21.0
20555,2013-11-14,10.0,64.00,71.516667,16.0
21000,2013-11-16,4.0,23.00,55.166667,6.0
21195,2013-11-18,4.0,34.25,49.116667,11.0


In [117]:
roll = test[time_lag_sums].rolling('7d', on = 'dt',closed = 'left')

In [116]:
test[['dt','assists','dk_score']].rolling('7d',on = 'dt').apply([np.sum,np.min])

/Users/shane/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  """Entry point for launching an IPython kernel.


TypeError: 'list' object is not callable

In [120]:
roll.apply(np.median)

/Users/shane/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  """Entry point for launching an IPython kernel.


,dt,made_field_goals,dk_score,minutes,rebounds
18249,2013-10-30,NaN,NaN,NaN,NaN
18571,2013-11-01,1.0,7.250,18.450000,3.0
18885,2013-11-03,1.5,14.375,20.666667,5.5
19365,2013-11-06,1.0,8.500,18.450000,4.0
19679,2013-11-08,2.0,21.500,20.166667,8.0
19970,2013-11-10,2.0,23.250,20.166667,9.0
20492,2013-11-13,2.0,23.250,20.166667,9.0
20555,2013-11-14,3.0,13.500,21.433333,4.0
21000,2013-11-16,1.0,7.000,19.566667,2.0
21195,2013-11-18,1.0,13.500,14.166667,4.0


In [76]:
player_hist = data[data['slug']==slug].iloc[1:,:]
player_hist.shape

(594, 54)

In [104]:
player = 'adamsst01'
kce = Key('slug').eq(player)
pe = "slug, #dt, minutes_06t"
ea = {"#dt":"date"}
resp = table.query(IndexName = 'slug-date-index', KeyConditionExpression = kce,
        ExpressionAttributeNames =ea,
        ProjectionExpression = pe,
        ScanIndexForward = False,
        Limit = 1
        )['Items']

In [110]:
'dt'  in resp[0].keys()

False

In [77]:
resp.index = resp['date']
lag_data = resp.sort_index(axis = 0,ascending=True).shift(1)
#lag_data = player_df.shift(1)
lag_data.head()

,assists_06,assists_per_min_06,attempted_field_goals_06,attempted_three_point_field_goals_06,blocks_06,blocks_per_min_06,date,dk_no_fg_06,dk_no_fg_per_min_06,dk_per_min_06,dk_score_06,doubles_06,dr_per_min_06,fg_pct_06,ft_pct_06,game_score_06,minutes_06,or_per_min_06,slug,steals_per_min_06,three_pct_06,turnovers_per_min_06,two_pct_06,w_06
date,,,,,,,,,,,,,,,,,,,,,,,,
2013-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-05,8,0.185,16,8,0,0,2013-01-02,21.5,0.498,1.286,55.5,1,0.116,0.688,0.75,28.4,43.15,0.023,curryst01,0.046,0.75,0.093,0.625,1
2013-01-09,6.5,0.179,13.5,6.5,0,0,2013-01-05,14,0.361,0.936,36.25,1,0.058,0.526,0.875,16.9,36.075,0.012,curryst01,0.023,0.475,0.081,0.562,0.5
2013-01-11,5.667,0.152,16.333,7,0,0,2013-01-09,13.667,0.348,0.945,37.167,1,0.063,0.502,0.583,15.867,37.556,0.016,curryst01,0.032,0.483,0.087,0.518,0.333
2013-01-13,7.25,0.183,17.75,7.25,0.25,0.006,2013-01-11,17.938,0.437,1.02,41.438,1.25,0.065,0.456,0.688,17.15,39.067,0.012,curryst01,0.041,0.456,0.071,0.46,0.5
